<a href="https://colab.research.google.com/github/JShyam33/Object-detection-on-video/blob/master/ob_dec_new_ssd_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!git clone https://github.com/tensorflow/models.git
!apt-get -qq install libprotobuf-java protobuf-compiler
!protoc ./models/research/object_detection/protos/string_int_label_map.proto --python_out=.
!cp -R models/research/object_detection/g3doc/ object_detection/
!rm -rf models

Cloning into 'models'...
remote: Enumerating objects: 23, done.
remote: Counting objects: 100% (23/23), done.
remote: Compressing objects: 100% (23/23), done.
remote: Total 28190 (delta 1), reused 1 (delta 0), pack-reused 28167
Receiving objects: 100% (28190/28190), 509.36 MiB | 35.23 MiB/s, done.
Resolving deltas: 100% (17347/17347), done.
Checking out files: 100% (3004/3004), done.


In [0]:
import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile

from distutils.version import StrictVersion
from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image
%pylab inline
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

import io
import base64
from IPython.display import HTML

import cv2
from google.colab.patches import cv2_imshow
vid_path = 'traffic2.mp4'
cap = cv2.VideoCapture(vid_path)
# This is needed since the notebook is stored in the object_detection folder.
sys.path.append("..")
from object_detection.utils import ops as utils_ops

#if StrictVersion(tf.__version__) < StrictVersion('1.12.0'):
 # raise ImportError('Please upgrade your TensorFlow installation to v1.12.*.')

Populating the interactive namespace from numpy and matplotlib


ModuleNotFoundError: ignored

In [0]:
# This is needed to display the images.
%matplotlib inline

In [0]:
from object_detection.utils import label_map_util

from object_detection.utils import visualization_utils as vis_util

In [0]:
# What model to download.
MODEL_NAME = 'ssd_mobilenet_v1_fpn_shared_box_predictor_640x640_coco14_sync_2018_07_03'
MODEL_FILE = MODEL_NAME + '.tar.gz'
DOWNLOAD_BASE = 'http://download.tensorflow.org/models/object_detection/'

# Path to frozen detection graph. This is the actual model that is used for the object detection.
PATH_TO_FROZEN_GRAPH = MODEL_NAME + '/frozen_inference_graph.pb'

# List of the strings that is used to add correct label for each box.
PATH_TO_LABELS = os.path.join('object_detection/data', 'mscoco_label_map.pbtxt')
print(PATH_TO_LABELS)

object_detection/data/mscoco_label_map.pbtxt


In [0]:
opener = urllib.request.URLopener()

opener.retrieve(DOWNLOAD_BASE + MODEL_FILE, MODEL_FILE)


tar_file = tarfile.open(MODEL_FILE)
for file in tar_file.getmembers():
  file_name = os.path.basename(file.name)
  if 'frozen_inference_graph.pb' in file_name:
    tar_file.extract(file, os.getcwd())


In [0]:
detection_graph = tf.Graph()
with detection_graph.as_default():
  od_graph_def = tf.GraphDef()
  with tf.gfile.GFile(PATH_TO_FROZEN_GRAPH, 'rb') as fid:
    serialized_graph = fid.read()
    od_graph_def.ParseFromString(serialized_graph)
    tf.import_graph_def(od_graph_def, name='')

In [0]:
category_index = label_map_util.create_category_index_from_labelmap(PATH_TO_LABELS, use_display_name=True)

W0626 12:30:15.103813 139931030505344 deprecation_wrapper.py:119] From /content/object_detection/utils/label_map_util.py:132: The name tf.gfile.GFile is deprecated. Please use tf.io.gfile.GFile instead.



In [0]:
def load_image_into_numpy_array(image):
  (im_width, im_height) = image.size
  return np.array(image.getdata()).reshape(
      (im_height, im_width, 3)).astype(np.uint8)

In [0]:
def run_inference_for_single_image(image, graph):
  with graph.as_default():
    with tf.Session() as sess:
      # Get handles to input and output tensors
      ops = tf.get_default_graph().get_operations()
      all_tensor_names = {output.name for op in ops for output in op.outputs}
      tensor_dict = {}
      for key in [
          'num_detections', 'detection_boxes', 'detection_scores',
          'detection_classes', 'detection_masks'
      ]:
        tensor_name = key + ':0'
        if tensor_name in all_tensor_names:
          tensor_dict[key] = tf.get_default_graph().get_tensor_by_name(
              tensor_name)
      if 'detection_masks' in tensor_dict:
        # The following processing is only for single image
        detection_boxes = tf.squeeze(tensor_dict['detection_boxes'], [0])
        detection_masks = tf.squeeze(tensor_dict['detection_masks'], [0])
        # Reframe is required to translate mask from box coordinates to image coordinates and fit the image size.
        real_num_detection = tf.cast(tensor_dict['num_detections'][0], tf.int32)
        detection_boxes = tf.slice(detection_boxes, [0, 0], [real_num_detection, -1])
        detection_masks = tf.slice(detection_masks, [0, 0, 0], [real_num_detection, -1, -1])
        detection_masks_reframed = utils_ops.reframe_box_masks_to_image_masks(
            detection_masks, detection_boxes, image.shape[1], image.shape[2])
        detection_masks_reframed = tf.cast(
            tf.greater(detection_masks_reframed, 0.5), tf.uint8)
        # Follow the convention by adding back the batch dimension
        tensor_dict['detection_masks'] = tf.expand_dims(
            detection_masks_reframed, 0)
      image_tensor = tf.get_default_graph().get_tensor_by_name('image_tensor:0')

      # Run inference
      output_dict = sess.run(tensor_dict,
                             feed_dict={image_tensor: image})

      # all outputs are float32 numpy arrays, so convert types as appropriate
      output_dict['num_detections'] = int(output_dict['num_detections'][0])
      output_dict['detection_classes'] = output_dict[
          'detection_classes'][0].astype(np.int64)
      output_dict['detection_boxes'] = output_dict['detection_boxes'][0]
      output_dict['detection_scores'] = output_dict['detection_scores'][0]
      if 'detection_masks' in output_dict:
        output_dict['detection_masks'] = output_dict['detection_masks'][0]
  return output_dict

In [0]:

out = cv2.VideoWriter('output2.avi',cv2.VideoWriter_fourcc(*'XVID'), 30, (int(cap.get(3)),int(cap.get(4))))



with detection_graph.as_default():
  with tf.Session(graph = detection_graph) as sess:
    while True:
      ret , image_np = cap.read()
      # Expand dimensions since the model expects images to have shape: [1, None, None, 3]
      if ret:
        
        image_np_expanded = np.expand_dims(image_np, axis=0)
        # Actual detection.
        output_dict = run_inference_for_single_image(image_np_expanded, detection_graph)
        # Visualization of the results of a detection.
        vis_util.visualize_boxes_and_labels_on_image_array(
            image_np,
            output_dict['detection_boxes'],
            output_dict['detection_classes'],
            output_dict['detection_scores'],
            category_index,
            instance_masks=output_dict.get('detection_masks'),
            use_normalized_coordinates=True,
            line_thickness=8)
        out.write(image_np)
        print('frame')
      else:
        out.release()
        cap.release()
        cv2.destroyAllWindows()
        print('failed')
        break



frame
frame
frame
frame
frame
frame
frame
frame
frame
frame
frame
frame
frame
frame
frame
frame
frame
frame
frame
frame
frame
frame
frame
frame
frame
frame
frame
frame
frame
frame
frame
frame
frame
frame
frame
frame
frame
frame
frame
frame
frame
frame
frame
frame
frame
frame
frame
frame
frame
frame
frame
frame
frame
frame
frame
frame
frame
frame
frame
frame
frame
frame
frame
frame
frame
frame
frame
frame
frame
frame
frame
frame
frame
frame
frame
frame
frame
frame
frame
frame
frame
frame
frame
frame
frame
frame
frame
frame
frame
frame
frame
frame
frame
frame
frame
frame
frame
frame
frame
frame
frame
frame
frame
frame
frame
frame
frame
frame
frame
frame
frame
frame
frame
frame
frame
frame
frame
frame
frame
frame
frame
frame
frame
frame
frame
frame
frame
frame
frame
frame
frame
frame
frame
frame
frame
frame
frame
frame
frame
frame
frame
frame
frame
frame
frame
frame
frame
frame
frame
frame
frame
frame
frame
frame
frame
frame
frame
frame
frame
frame
frame
frame
frame
frame
frame
frame
fram